# Feature concatenation for Persuasive Essays dataset

Feature concatenation for the Persuasive Essays dataset where the numerical structural features are concatenated with the BERT embedding.

## Packages

In [ ]:
!pip install pandas==1.3.4
!pip install transformers==4.12.5
!pip install datasets==1.15.1
!pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


## Imports

In [ ]:
import os
import pickle

from collections import Counter

from sklearn.metrics import classification_report

import numpy as np
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss

import transformers
from transformers import Trainer
from transformers import BertTokenizer
from transformers import BertModel, BertPreTrainedModel
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers.data.data_collator import DataCollatorWithPadding

import datasets
from datasets import Dataset
from datasets import ClassLabel
from datasets import load_metric

## File and folder paths

In [ ]:
# Your paths go here:

DATA_FILE = '../datasets/dataset_persuasive_essays_icann.pt'
RESULTS_FOLDER = '../results/'

## GPU

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

device(type='cuda')

## Load dataset

In [ ]:
dataset = torch.load(DATA_FILE)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['split', 'text', 'labels', 'sentence', 'topic_and_full_sentence', 'topic_full_sentence_stuctural_fts', 'topic_full_sentence_structural_fts_combined', 'feature_tensor'],
        num_rows: 4709
    })
    test: Dataset({
        features: ['split', 'text', 'labels', 'sentence', 'topic_and_full_sentence', 'topic_full_sentence_stuctural_fts', 'topic_full_sentence_structural_fts_combined', 'feature_tensor'],
        num_rows: 1258
    })
})

## Tokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
label_names = set(dataset['train']['labels'])
label_nb = len(label_names)
labels = ClassLabel(num_classes=label_nb, names=label_names)

In [ ]:
labels

ClassLabel(num_classes=3, names={'MajorClaim', 'Claim', 'Premise'}, names_file=None, id=None)

In [ ]:
def tokenize(batch):
    tokens = tokenizer(batch['topic_and_full_sentence'], truncation=True, padding=True, max_length=512)
    tokens['labels'] = labels.str2int(batch['labels'])
    return tokens

In [ ]:
dataset = dataset.map(tokenize, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels', 'feature_tensor'])

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'feature_tensor', 'input_ids', 'labels', 'sentence', 'split', 'text', 'token_type_ids', 'topic_and_full_sentence', 'topic_full_sentence_structural_fts_combined', 'topic_full_sentence_stuctural_fts'],
        num_rows: 4709
    })
    test: Dataset({
        features: ['attention_mask', 'feature_tensor', 'input_ids', 'labels', 'sentence', 'split', 'text', 'token_type_ids', 'topic_and_full_sentence', 'topic_full_sentence_structural_fts_combined', 'topic_full_sentence_stuctural_fts'],
        num_rows: 1258
    })
})

## Train-test split

In [ ]:
train_dataset = dataset['train'].shuffle(seed=42)
test_dataset = dataset['test'].shuffle(seed=42)

train_val_datasets = dataset['train'].train_test_split(train_size=0.8)
train_dataset = train_val_datasets['train']
val_dataset = train_val_datasets['test']

In [ ]:
dataset_d = {}
dataset_d['train'] = train_dataset
dataset_d['test'] = test_dataset
dataset_d['val'] = val_dataset

## Model

In [ ]:
NUM_LABELS = labels.num_classes
NUM_FEATURES = dataset_d['train']['feature_tensor'].shape[1]
BATCH_SIZE = 16
NB_EPOCHS = 8

In [ ]:
NUM_LABELS, NUM_FEATURES

(3, 5)

In [ ]:
class CustomBertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config, num_features):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.num_features = num_features # *** MODIF ***
        
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size + num_features, config.num_labels) # *** MODIF ***

        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        feature_tensor=None # *** MODIF ***
    ):
        
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`, defaults to :obj:`None`):
            Labels for computing the sequence classification/regression loss.
            Indices should be in :obj:`[0, ..., config.num_labels - 1]`.
            If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).

    Returns:
        :obj:`tuple(torch.FloatTensor)` comprising various elements depending on the configuration (:class:`~transformers.BertConfig`) and inputs:
        loss (:obj:`torch.FloatTensor` of shape :obj:`(1,)`, `optional`, returned when :obj:`label` is provided):
            Classification (or regression if config.num_labels==1) loss.
        logits (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, config.num_labels)`):
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        hidden_states (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``config.output_hidden_states=True``):
            Tuple of :obj:`torch.FloatTensor` (one for the output of the embeddings + one for the output of each layer)
            of shape :obj:`(batch_size, sequence_length, hidden_size)`.

            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        attentions (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``config.output_attentions=True``):
            Tuple of :obj:`torch.FloatTensor` (one for each layer) of shape
            :obj:`(batch_size, num_heads, sequence_length, sequence_length)`.

            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention
            heads.

    Examples::

        from transformers import BertTokenizer, BertForSequenceClassification
        import torch

        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

        input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids, labels=labels)

        loss, logits = outputs[:2]

        """

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        pooled_output = outputs[1]
        pooled_output = torch.cat([pooled_output, feature_tensor], dim=1) # *** MODIF *** COMMENT FOR DUMMY EXPERIMENT
        pooled_output = self.dropout(pooled_output)
                
        logits = self.classifier(pooled_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [ ]:
model = CustomBertForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                            num_labels=NUM_LABELS, 
                                                            num_features=NUM_FEATURES)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing CustomBertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBertForSequenceClassification were not initialized from

CustomBertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

## Trainer

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs[1]#.get('logits')
        loss_fct = nn.CrossEntropyLoss()#(weight=class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
metric = load_metric('f1')

def compute_metrics(eval_pred):
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    return metric.compute(predictions=predictions, references=labels, average='macro')

In [ ]:
training_args = TrainingArguments(
    
    # output
    output_dir=RESULTS_FOLDER,          
    
    # params
    num_train_epochs=NB_EPOCHS,               # nb of epochs
    per_device_train_batch_size=BATCH_SIZE,   # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,    # cf. paper Sun et al.
    learning_rate=5e-3,#2e-5,                 # cf. paper Sun et al.
#     warmup_steps=500,                         # number of warmup steps for learning rate scheduler
    warmup_ratio=0.1,                         # cf. paper Sun et al.
    weight_decay=0.01,                        # strength of weight decay
    
    # eval
    evaluation_strategy="steps",              # cf. paper Sun et al.
    eval_steps=20,                            # cf. paper Sun et al.
    
    # log
    logging_dir=RESULTS_FOLDER+'logs',  
    logging_strategy='steps',
    logging_steps=20,
    
    # save
    save_strategy='steps',
    save_total_limit=2,
    # save_steps=20, # default 500
    load_best_model_at_end=True,              # cf. paper Sun et al.
    # metric_for_best_model='eval_loss'
    metric_for_best_model='f1'
)

In [ ]:
trainer = CustomTrainer( # Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

In [ ]:
results = trainer.train()

The following columns in the training set  don't have a corresponding argument in `CustomBertForSequenceClassification.forward` and have been ignored: text, topic_full_sentence_stuctural_fts, sentence, split, topic_and_full_sentence, topic_full_sentence_structural_fts_combined.
***** Running training *****
  Num examples = 3767
  Num Epochs = 8
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1888


Step,Training Loss,Validation Loss,F1
20,1.013200,0.934032,0.255672
40,0.975100,0.977639,0.255672
60,0.974400,1.131830,0.128535
80,1.030400,0.935517,0.255672
100,1.063400,0.895113,0.255672
120,0.883800,1.003420,0.255672
140,0.971300,1.385089,0.342617
160,1.157800,1.080452,0.255672
180,1.033700,1.493649,0.255672
200,1.466200,1.311440,0.128535


The following columns in the evaluation set  don't have a corresponding argument in `CustomBertForSequenceClassification.forward` and have been ignored: text, topic_full_sentence_stuctural_fts, sentence, split, topic_and_full_sentence, topic_full_sentence_structural_fts_combined.
***** Running Evaluation *****
  Num examples = 942
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `CustomBertForSequenceClassification.forward` and have been ignored: text, topic_full_sentence_stuctural_fts, sentence, split, topic_and_full_sentence, topic_full_sentence_structural_fts_combined.
***** Running Evaluation *****
  Num examples = 942
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `CustomBertForSequenceClassification.forward` and have been ignored: text, topic_full_sentence_stuctural_fts, sentence, split, topic_and_full_sentence, topic_full_sentence_structural_fts_combined.
***** Running Eva

## Results

In [ ]:
model.eval()

CustomBertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [ ]:
test_trainer = Trainer(model, data_collator=DataCollatorWithPadding(tokenizer))
test_raw_preds, test_labels, _ = test_trainer.predict(test_dataset)
test_preds = np.argmax(test_raw_preds, axis=1)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set  don't have a corresponding argument in `CustomBertForSequenceClassification.forward` and have been ignored: text, topic_full_sentence_stuctural_fts, sentence, split, topic_and_full_sentence, topic_full_sentence_structural_fts_combined.
***** Running Prediction *****
  Num examples = 1258
  Batch size = 8


In [ ]:
target_name = labels.int2str([0,1,2])
print(classification_report(test_labels, test_preds, target_names=target_name))

              precision    recall  f1-score   support

  MajorClaim       0.70      0.99      0.82       152
       Claim       0.67      0.50      0.57       301
     Premise       0.89      0.90      0.90       805

    accuracy                           0.82      1258
   macro avg       0.75      0.80      0.76      1258
weighted avg       0.81      0.82      0.81      1258

